In [3]:
import tweepy
import csv
import pandas as pd
import sys
from textblob import TextBlob
import re
import json

In [6]:
####chiavi Jacopo
consumer_key = 'consumer_key'
consumer_secret = 'consumer_secret'
access_token = 'access_token'
access_token_secret = 'access_token_secret'

In [7]:
#autenticazione
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [18]:
csvFile = open('Moderna_2502.csv', 'a') #file in cui salvare i tweet
csvWriter = csv.writer(csvFile)

#Scarico i tweet che contengono la parola 'astrazeneca'
#fromDate e toDate format "YYYYMMDDhhmm"
#cambiare date e numero di items

#link per le varie flag:
#https://developer.twitter.com/en/docs/twitter-api/enterprise/rules-and-filtering/operators-by-product

#ho messo lingua inglese

#ci sono 4 casi: tweet - tweet esteso - retweet - retweet esteso 
#sfortunatamente per avere il testo completo ci sono 4 "percorsi" diversi, quindi ho distinto i casi

for tweet in tweepy.Cursor(api.search_30_day, environment_name = 'dslab', query="Moderna lang:it", fromDate="202102251100", toDate = "202102252000").items(1000):
    
    if (hasattr(tweet, 'retweeted_status')): #se è retweet
        
        if (hasattr(tweet.retweeted_status, 'extended_tweet')): #se è esteso
            csvWriter.writerow([True, tweet.created_at, tweet.user.screen_name, #True(è retweet), #data, #user
                                tweet.retweeted_status.extended_tweet['full_text'].replace("\n", ""), #testo
                                tweet.retweet_count]) #numero di retweet
        else: #se non è esteso
            csvWriter.writerow([True, tweet.created_at, tweet.user.screen_name, #True(retweet), #data, #user
                                tweet.retweeted_status.text.replace("\n", ""), #testo
                                tweet.retweet_count]) #numero di retweet   
    
    else: #se non è retweet
        
        if (hasattr(tweet, 'extended_tweet')): #se è esteso
            csvWriter.writerow([False, tweet.created_at, tweet.user.screen_name, #False(non è retweet), #data, #user
                                tweet.extended_tweet['full_text'].replace("\n", ""), #testo
                                tweet.retweet_count]) #numero di retweet
        else: #se non è esteso
            csvWriter.writerow([False, tweet.created_at, tweet.user.screen_name, #True(retweet), #data, #user
                                tweet.text.replace("\n", ""), #testo
                                tweet.retweet_count]) #numero di retweet    

        
csvFile.close()

In [19]:
colnames=['retweet', 'data', 'user', 'testo', 'retweet count'] 
df = pd.read_csv('Moderna_2502.csv', encoding='utf-8', header = None, names=colnames)
df

#2021-03-14 19:15:40

,retweet,data,user,testo,retweet count
0,False,2021-02-24 19:59:34,WalTer62851003,@AngryMercurio @Boborius @maiunfatto @perchete...,0
1,False,2021-02-24 19:58:49,Francescovotta1,"@f_venere @neXtquotidiano È una visione, che s...",0
2,True,2021-02-24 19:58:18,iri_osho,park jimin top student del dipartimento di dan...,0
3,True,2021-02-24 19:51:12,_joes__,park jimin top student del dipartimento di dan...,0
4,True,2021-02-24 19:49:01,lachimolala_sof,park jimin top student del dipartimento di dan...,0
...,...,...,...,...,...
977,False,2021-02-24 06:31:19,habitxsunshine,Cami be like:Pichula triste Pichula moderna Pi...,0
978,True,2021-02-24 06:31:00,RANRoberto,"L’#UE non è fallita finora per nessuna crisi, ...",0
979,True,2021-02-24 06:28:12,giacomo_novara,"“Per paura di morire,avete smesso di vivere”- ...",0
980,True,2021-02-24 06:28:11,pma85,"Stamattina indecisa tra ansia classica, modern...",0


In [20]:
df.sort_values('data')

,retweet,data,user,testo,retweet count
981,True,2021-02-24 06:26:47,checovenier,Le missioni Apollo hanno profanato la Luna… no...,0
980,True,2021-02-24 06:28:11,pma85,"Stamattina indecisa tra ansia classica, modern...",0
979,True,2021-02-24 06:28:12,giacomo_novara,"“Per paura di morire,avete smesso di vivere”- ...",0
978,True,2021-02-24 06:31:00,RANRoberto,"L’#UE non è fallita finora per nessuna crisi, ...",0
977,False,2021-02-24 06:31:19,habitxsunshine,Cami be like:Pichula triste Pichula moderna Pi...,0
...,...,...,...,...,...
488,True,2021-02-24 23:39:33,genovesergio76,"Moderna, un vaccino contro la variante sudafri...",0
487,True,2021-02-24 23:41:39,genovesergio76,Moderna inizia i test clinici del vaccino cont...,0
486,True,2021-02-24 23:46:02,MarcoZanolini2,"Moderna, un vaccino contro la variante sudafri...",0
485,False,2021-02-24 23:50:15,parkhyunso,"la voglio super luminosa, moderna, grande, pos...",1


In [3]:
import os
import glob
import pandas as pd
os.chdir('/path')

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [4]:
combined_csv

,Unnamed: 0,retweet,data,user,testo,retweet count,english,polarity
0,0,True,2021-03-01 19:59:04,Paolo18030138,FacciamoRete FacciamoInformazione Coronavirus ...,0,Let's make coronavirus covid19 vaccinh24 fuckc...,0.200000
1,1,True,2021-03-01 19:58:45,SaverioMascaro,AstraZenecaPfizerModerna ora Johnsonandjohnson...,0,Astrazenecapfizermoderna now Johnsonandjohnson...,0.200000
2,2,True,2021-03-01 19:58:13,BigazziMarco,FacciamoRete FacciamoInformazione Coronavirus ...,0,Let's make coronavirus covid19 vaccinh24 fuckc...,0.200000
3,3,False,2021-03-01 19:58:08,liliaragnar,AstraZenecaPfizerModerna ora Johnsonandjohnson...,9,Astrazenecapfizermoderna now Johnsonandjohnson...,0.200000
4,4,False,2021-03-01 19:57:53,FirenzePost,Vaccini: Ue acquista altre 300 milioni di dosi...,0,Vaccines: EU buy more 300 million doses of mod...,0.350000
...,...,...,...,...,...,...,...,...
772,772,True,2021-03-25 00:33:18,Faniuzza1,"""Pfizer e Moderna danno una mazzata cosmica do...",0,"""Pfizer and modern damage a cosmic blow after ...",0.070833
773,773,True,2021-03-25 00:27:19,ZombieBuster5,"""Pfizer e Moderna danno una mazzata cosmica do...",0,"""Pfizer and modern damage a cosmic blow after ...",0.070833
774,774,False,2021-03-25 00:26:52,BarbaraRaval,"""Pfizer e Moderna danno una mazzata cosmica do...",62,"""Pfizer and modern damage a cosmic blow after ...",0.070833
775,775,True,2021-03-25 00:23:30,DesiMalafronte,serena che pur essendo una ballerina moderna r...,0,serene that despite being a modern dancer mana...,0.200000
